In [50]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error

In [51]:
import numpyro
import numpy as np 

import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

In [52]:
garstec_data = 'C:\\Users\\Dell\\Downloads\\Garstec_AS09_chiara.hdf5'

# 7 Inputs
ages = []
massini = []
fehini = []
alphamlt = []
yini = []
eta = []
alphafe = []

# 8 Outputs
teff = []
luminosity = []
dnufit = []
FeH = []
G_GAIA = []
massfin = []
numax = []
MeH = []

# Open the hdf5 file (read-only mode)
with h5py.File(garstec_data, 'r') as hdf:

    grid = hdf['grid']
    tracks = grid['tracks']

    # Get a list of track names and shuffle for random sampling
    track_names = list(tracks.keys())
    random.seed(1)
    random.shuffle(track_names)

    # Choose a subset of tracks to process
    num_tracks = 1000  # Set the number of tracks to process
    selected_tracks = track_names[:num_tracks]

    for track_name in selected_tracks:  # Iterate over the selected track names
        track = tracks[track_name]
        
        # Inputs
        ages.append(track['age'][:])
        massini.append(track['massini'][:])
        fehini.append(track['FeHini'][:])
        alphamlt.append(track['alphaMLT'][:])
        yini.append(track['yini'][:])
        eta.append(track['eta'][:])
        alphafe.append(track['alphaFe'][:])

        # Outputs
        teff.append(track['Teff'][:])
        luminosity.append(track['LPhot'][:])
        dnufit.append(track['dnufit'][:])
        FeH.append(track['FeH'][:])
        G_GAIA.append(track['G_GAIA'][:])
        massfin.append(track['massfin'][:])
        numax.append(track['numax'][:])
        MeH.append(track['MeH'][:])

# Convert lists to numpy arrays and concatenate them (make one big list)

# Define a small constant to avoid log10(0)
epsilon = 1e-10

# Features requiring log10 transformation
log10_vars_inputs = [ages, massini, alphamlt, eta, alphafe]

# Transform log10 variables
log10_transformed_inputs = [np.log10(np.maximum(np.concatenate(var).reshape(-1, 1), epsilon)) for var in log10_vars_inputs]

# Concatenate all inputs, including raw `fehini` and `yini`
inputs = np.hstack(log10_transformed_inputs + [np.concatenate(fehini).reshape(-1, 1), np.concatenate(yini).reshape(-1, 1)])

# Features requiring log10 transformation (strictly positive outputs)
log10_vars_outputs = [teff, luminosity, dnufit, G_GAIA, massfin, numax]

# Transform log10 variables
log10_transformed_outputs = [np.log10(np.maximum(np.concatenate(var).reshape(-1, 1), epsilon)) for var in log10_vars_outputs]

# Combine transformed log10 outputs with raw FeH and MeH
# FeH and MeH are not transformed, concatenated directly
outputs = np.hstack(log10_transformed_outputs + [np.concatenate(FeH).reshape(-1, 1), np.concatenate(MeH).reshape(-1, 1)])



In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.FloatTensor(y_train)
y_test_tensor = torch.FloatTensor(y_test)

dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  


In [53]:
# Define the neural network
class GarstecNet(nn.Module):
    def __init__(self):
        super(GarstecNet, self).__init__()
        self.dense1 = nn.Linear(7, 64)   # Input layer
        self.dense2 = nn.Linear(64, 64)
        self.dense3 = nn.Linear(64, 64)  
        self.dense4 = nn.Linear(64, 64)
        self.dense5 = nn.Linear(64, 64)
        self.dense6 = nn.Linear(64, 8)    # Output layer
        

    def forward(self, x):
        x = torch.relu(self.dense1(x))
        x = torch.relu(self.dense2(x))
        x = torch.relu(self.dense3(x))  
        x = torch.relu(self.dense4(x))
        x = torch.relu(self.dense5(x))
        x = self.dense6(x)
        return x

    
# Load the pre-trained model weights
model = GarstecNet()
model.load_state_dict(torch.load('garstec_model_V3_state.pth'))

# Set the model to evaluation mode
model.eval()
print("Model loaded successfully.")

Model loaded successfully.


C:\Users\Dell\AppData\Local\Temp\ipykernel_34276\3765155684.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('garstec_model_V3_state.pth

In [54]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())
    y_test_actual = scaler_y.inverse_transform(y_test_tensor.numpy())

    # Calculate evaluation metrics
    r2 = r2_score(y_test_actual, predictions)
    mae = mean_absolute_error(y_test_actual, predictions)

    print(f'R^2 Score: {r2:.4f}')
    print(f'Mean Absolute Error: {mae:.4f}')

R^2 Score: 0.9806
Mean Absolute Error: 0.0856


In [ ]:
# Fixed number of stars
n_stars = 5

# Dictionary to store predictions
predictions = {
    'Teff': [],
    'Luminosity': [],
    'Dnufit': [],
    'FeH': [],
    'G_GAIA': [],
    'Massfin': [],
    'Numax': [],
    'MeH': []
}

with h5py.File(garstec_data, 'r') as hdf:
    for _ in range(n_stars):
        # Select a random track
        specific_track_name = np.random.choice(selected_tracks)
        specific_track = hdf['grid']['tracks'][specific_track_name]
        
        # Randomly sample a single data point from the track
        # This is because with too many points the model couldnt "walk" far enough so simplified it while keeping the
        # cluster of stars property of the problem.
        num_points = len(specific_track['age'])
        idx = np.random.randint(0, num_points)  # Random index
        
        
        # Retrieve features for the sampled point
        ages = specific_track['age'][idx].reshape(-1, 1)
        massini = specific_track['massini'][idx].reshape(-1, 1)
        fehini = specific_track['FeHini'][idx].reshape(-1, 1)
        alphamlt = specific_track['alphaMLT'][idx].reshape(-1, 1)
        yini = specific_track['yini'][idx].reshape(-1, 1)
        eta = specific_track['eta'][idx].reshape(-1, 1)
        alphafe = specific_track['alphaFe'][idx].reshape(-1, 1)
        
        # Combine features into a single array
        epsilon = 1e-10
        log10_vars_inputs = [ages, massini, alphamlt, eta, alphafe]
        log10_transformed_inputs = [np.log10(np.maximum(var, epsilon)) for var in log10_vars_inputs]
        all_features = np.hstack(log10_transformed_inputs + [fehini, yini])

        # Scale features
        all_features_scaled = scaler_X.transform(all_features)
        all_features_tensor = torch.FloatTensor(all_features_scaled)
        
        # Make predictions
        model.eval()
        with torch.no_grad():
            predictions_specific = model(all_features_tensor).numpy()
            predictions_specific = scaler_y.inverse_transform(predictions_specific)


        # Extract and store predictions for this star
        predictions['Teff'].append(10**predictions_specific[0, 0])
        predictions['Luminosity'].append(10**predictions_specific[0, 1])
        predictions['Dnufit'].append(10**predictions_specific[0, 2])
        predictions['FeH'].append(10**predictions_specific[0, 3])
        predictions['G_GAIA'].append(10**predictions_specific[0, 4])
        predictions['Massfin'].append(10**predictions_specific[0, 5])
        predictions['Numax'].append(10**predictions_specific[0, 6])
        predictions['MeH'].append(10**predictions_specific[0, 7])

teff_obs = np.array(predictions['Teff'])
luminosity_obs = np.array(predictions['Luminosity'])
dnufit_obs = np.array(predictions['Dnufit'])
FeH_obs = np.array(predictions['FeH'])
G_GAIA_obs = np.array(predictions['G_GAIA'])
massfin_obs = np.array(predictions['Massfin'])
numax_obs = np.array(predictions['Numax'])
MeH_obs = np.array(predictions['MeH'])

# Checking data points for debugging
print(teff_obs)
print(luminosity_obs)
print(dnufit_obs)
print(FeH_obs)
print(G_GAIA_obs)
print(massfin_obs)
print(numax_obs)
print(MeH_obs)


[4932.64058133 4652.25822784 4891.95228153 5531.54853543 5091.07530194]
[ 11.13728656  95.38456881   8.33429052  12.52324302 107.52646291]
[11.68068749  2.61558766 16.65392518 16.70058139  2.85426828]
[2.19656126e+00 1.28975863e-07 1.73269807e+00 1.46599437e+00
 1.03831424e-10]
[0.73441414 1.3223209  0.97818339 0.90089512 1.10754626]
[0.03799601 0.00650092 0.06558518 0.06217756 0.00663714]
[0.2827621  0.31256809 0.31589133 0.12381988 0.01711447]
[0.31501224 0.31794541 0.24498929 0.16543927 0.05322677]


In [85]:
def hierarchical_model(teff_obs, luminosity_obs, dnufit_obs, FeH_obs, 
                       G_GAIA_obs, massfin_obs, numax_obs, MeH_obs):
    # Hyperpriors for group-level parameters
    # Picked based on a brief look at the data in order to get the model working, more thought will have to be
    # given to these, especially the smaller ones such as FeH etc
    mean_teff = numpyro.sample('mean_teff', dist.Normal(6000, 500))
    sigma_teff = numpyro.sample('sigma_teff', dist.HalfNormal(200))

    mean_luminosity = numpyro.sample('mean_luminosity', dist.Normal(80, 20))
    sigma_luminosity = numpyro.sample('sigma_luminosity', dist.HalfNormal(10))

    mean_dnufit = numpyro.sample('mean_dnufit', dist.Normal(20, 5))
    sigma_dnufit = numpyro.sample('sigma_dnufit', dist.HalfNormal(2))

    mean_FeH = numpyro.sample('mean_FeH', dist.Normal(0, 0.2))
    sigma_FeH = numpyro.sample('sigma_FeH', dist.HalfNormal(0.1))

    mean_G_GAIA = numpyro.sample('mean_G_GAIA', dist.Normal(0.8, 0.05))
    sigma_G_GAIA = numpyro.sample('sigma_G_GAIA', dist.HalfNormal(0.02))

    mean_massfin = numpyro.sample('mean_massfin', dist.Normal(0.1, 0.05))
    sigma_massfin = numpyro.sample('sigma_massfin', dist.HalfNormal(0.02))

    mean_numax = numpyro.sample('mean_numax', dist.Normal(0.023, 0.002))
    sigma_numax = numpyro.sample('sigma_numax', dist.HalfNormal(0.001))

    mean_MeH = numpyro.sample('mean_MeH', dist.Normal(0.025, 0.1))
    sigma_MeH = numpyro.sample('sigma_MeH', dist.HalfNormal(0.05))

    # Star-level distributions 
    # Sampling the stars based off of the hyper priors above, using the numpyro plate feature
    with numpyro.plate('stars', n_stars):
        teff = numpyro.sample('teff', dist.Normal(mean_teff, sigma_teff))
        luminosity = numpyro.sample('luminosity', dist.Normal(mean_luminosity, sigma_luminosity))
        dnufit = numpyro.sample('dnufit', dist.Normal(mean_dnufit, sigma_dnufit))
        FeH = numpyro.sample('FeH', dist.Normal(mean_FeH, sigma_FeH))
        G_GAIA = numpyro.sample('G_GAIA', dist.Normal(mean_G_GAIA, sigma_G_GAIA))
        massfin = numpyro.sample('massfin', dist.Normal(mean_massfin, sigma_massfin))
        numax = numpyro.sample('numax', dist.Normal(mean_numax, sigma_numax))
        MeH = numpyro.sample('MeH', dist.Normal(mean_MeH, sigma_MeH))

        # Observation noise
        obs_sigma = 5.0  # Picked as a starting point but more thought needs to be given
        numpyro.sample('obs_teff', dist.Normal(teff, obs_sigma), obs=teff_obs)
        numpyro.sample('obs_luminosity', dist.Normal(luminosity, obs_sigma), obs=luminosity_obs)
        numpyro.sample('obs_dnufit', dist.Normal(dnufit, obs_sigma), obs=dnufit_obs)
        numpyro.sample('obs_FeH', dist.Normal(FeH, obs_sigma), obs=FeH_obs)
        numpyro.sample('obs_G_GAIA', dist.Normal(G_GAIA, obs_sigma), obs=G_GAIA_obs)
        numpyro.sample('obs_massfin', dist.Normal(massfin, obs_sigma), obs=massfin_obs)
        numpyro.sample('obs_numax', dist.Normal(numax, obs_sigma), obs=numax_obs)
        numpyro.sample('obs_MeH', dist.Normal(MeH, obs_sigma), obs=MeH_obs)


In [99]:
from jax import random
from numpyro.infer import NUTS, MCMC

# Set up the NUTS kernel and MCMC
nuts_kernel = NUTS(hierarchical_model)
hbm_mcmc = MCMC(nuts_kernel, num_samples=8000, num_warmup=8000, num_chains=2, thinning=4)

# Random seed
rng_key = random.PRNGKey(0)

# Run MCMC with the observed data
hbm_mcmc.run(
    rng_key,
    teff_obs=teff_obs,
    luminosity_obs=luminosity_obs,
    dnufit_obs=dnufit_obs,
    FeH_obs=FeH_obs,
    G_GAIA_obs=G_GAIA_obs,
    massfin_obs=massfin_obs,
    numax_obs=numax_obs,
    MeH_obs=MeH_obs
)

# Extract posterior samples
posterior_samples = hbm_mcmc.get_samples()


C:\Users\Dell\AppData\Local\Temp\ipykernel_34276\2889296754.py:6: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  hbm_mcmc = MCMC(nuts_kernel, num_samples=8000, num_warmup=8000, num_chains=2, thinning=4)
sample: 100%|██████████| 16000/16000 [06:09<00:00, 43.32it/s, 255 steps of size 2.57e-02. acc. prob=0.68] 


In [100]:
hbm_mcmc.print_summary()


                        mean       std    median      5.0%     95.0%     n_eff     r_hat
            FeH[0]     -0.01      0.22     -0.02     -0.35      0.38    701.86      1.00
            FeH[1]     -0.01      0.22     -0.02     -0.35      0.35    456.33      1.01
            FeH[2]     -0.01      0.22     -0.02     -0.34      0.35    342.10      1.01
            FeH[3]     -0.01      0.22     -0.02     -0.40      0.35    634.49      1.00
            FeH[4]     -0.00      0.21     -0.02     -0.34      0.37   1045.07      1.00
         G_GAIA[0]      0.80      0.05      0.79      0.71      0.88   1803.28      1.00
         G_GAIA[1]      0.80      0.05      0.79      0.72      0.89    446.38      1.01
         G_GAIA[2]      0.80      0.05      0.79      0.72      0.89   1782.53      1.00
         G_GAIA[3]      0.79      0.05      0.79      0.71      0.89    528.98      1.01
         G_GAIA[4]      0.80      0.05      0.79      0.72      0.89    721.53      1.00
            MeH[0]  